# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool

In [3]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name

'search'

In [5]:
search.description

'Search for weather online'

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [7]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [8]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [9]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [10]:
search.run("sf")

'42f'

In [70]:
from pydantic import BaseModel, Field

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    import requests
    from datetime import datetime, UTC

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
        "timezone": "UTC",  # helps make returned times consistent
    }

    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    results = response.json()

    current_utc_time = datetime.now(UTC)

    def _parse_to_utc(ts: str) -> datetime:
        # Handle "Z" suffix if present
        dt = datetime.fromisoformat(ts.replace("Z", "+00:00"))
        # If Open-Meteo returned a naive timestamp, assume it's UTC
        if dt.tzinfo is None:
            return dt.replace(tzinfo=UTC)
        # Otherwise normalize to UTC
        return dt.astimezone(UTC)

    time_list = [_parse_to_utc(t) for t in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]

    closest_time_index = min(
        range(len(time_list)),
        key=lambda i: abs(time_list[i] - current_utc_time),
    )

    current_temperature = temperature_list[closest_time_index]
    return f"The current temperature is {current_temperature}°C"

In [71]:
get_current_temperature.name

'get_current_temperature'

In [72]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [73]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

In [74]:
from langchain_core.utils.function_calling import convert_to_openai_tool

In [75]:
convert_to_openai_tool(get_current_temperature)

{'type': 'function',
 'function': {'name': 'get_current_temperature',
  'description': 'Fetch current temperature for given coordinates.',
  'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
     'type': 'number'},
    'longitude': {'description': 'Longitude of the location to fetch weather data for',
     'type': 'number'}},
   'required': ['latitude', 'longitude'],
   'type': 'object'}}}

In [76]:
get_current_temperature.invoke({"latitude": 13, "longitude": 14})

'The current temperature is 21.2°C'

In [77]:

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    import wikipedia
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [78]:
search_wikipedia.name

'search_wikipedia'

In [79]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [80]:
convert_to_openai_tool(search_wikipedia)

{'type': 'function',
 'function': {'name': 'search_wikipedia',
  'description': 'Run Wikipedia search and get page summaries.',
  'parameters': {'properties': {'query': {'type': 'string'}},
   'required': ['query'],
   'type': 'object'}}}

In [81]:
search_wikipedia.invoke({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Vector database\nSummary: A vector database, vector store or vector search engine is a database that stores and retrieves embeddings of data in vector space. Vector databases typically implement approximate nearest neighbor algorithms so users can search for records semantically similar to a given input, unlike traditional databases which primarily look up records by exact match. Use-cases for vector databases include similarity search, semantic search, multi-modal search, recommendations engines, object detection, and retrieval-augmented generation (RAG).\nVector embeddings are mathematical representations of data in a high-dime

## REQUIRES CONVERTING

In [82]:
import json

from langchain_openai import ChatOpenAI

from langchain_community.agent_toolkits.json.toolkit import JsonToolkit
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain_community.utilities.requests import TextRequestsWrapper


In [83]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [84]:
# Parse the OpenAPI spec (as JSON here) and wrap it in a JsonSpec for the toolkit.
spec_dict = json.loads(text)
json_spec = JsonSpec(dict_=spec_dict, max_value_length=4000)

# Requests wrapper (no auth needed for the Petstore example).
requests_wrapper = TextRequestsWrapper(headers={})


In [85]:
# Build a modern set of Tools for interacting with an OpenAPI-described API.
#
# NOTE: OpenAPIToolkit.get_tools() includes a `json_explorer` tool implemented as an *agent*.
# That agent internally uses stop sequences, which GPT-5-mini rejects (no `stop` support).
# To stay fully modern (tools-first) *and* compatible with GPT-5-mini, we compose:
#   - RequestsToolkit: deterministic HTTP request tools (GET/POST/PATCH/PUT/DELETE)
#   - JsonToolkit: deterministic tools for exploring the OpenAPI JSON spec
# This avoids any legacy agent/function-calling shims and avoids passing `stop`.

llm = ChatOpenAI(model="gpt-5-mini", disabled_params={"stop": None})

requests_toolkit = RequestsToolkit(
    requests_wrapper=requests_wrapper,
    allow_dangerous_requests=True,
)
json_toolkit = JsonToolkit(spec=json_spec)

openapi_tools = [*requests_toolkit.get_tools(), *json_toolkit.get_tools()]
[len(openapi_tools), [t.name for t in openapi_tools]]


[7,
 ['requests_get',
  'requests_post',
  'requests_patch',
  'requests_put',
  'requests_delete',
  'json_spec_list_keys',
  'json_spec_get_value']]

In [86]:
# Bind the OpenAPI tools to the model (OpenAI "tools" schema).
llm_with_openapi_tools = llm.bind_tools(openapi_tools)


In [87]:
llm_with_openapi_tools.invoke("What are three pet names? (Call the API if you need to.)")


AIMessage(content='Here are three pet names: Bella, Max, Luna.\n\nWant names for a specific type (dog, cat, bird) or personality?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 706, 'total_tokens': 807, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ctj6p9BHTmSokKljRABR4U3etmftS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b8119-4009-7562-be6a-b43bf8f937fc-0', usage_metadata={'input_tokens': 706, 'output_tokens': 101, 'total_tokens': 807, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [88]:
llm_with_openapi_tools.invoke("Tell me about the pet with id 42. (Use the API.)")


AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 706, 'total_tokens': 804, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ctj6sG7Y4dwEJBwrDuOZTdIOdJxUv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8119-50ed-7fa2-be6a-55858d1c81fe-0', tool_calls=[{'name': 'requests_get', 'args': {'url': 'https://petstore.swagger.io/v2/pet/42'}, 'id': 'call_j4vZwzz28FZbQKomLZg1Q6Uw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 706, 'output_tokens': 98, 'total_tokens': 804, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [89]:
# If the model chose to call a tool, you'll see tool_calls on the AIMessage.
msg = llm_with_openapi_tools.invoke("Look up the pet with id 42 and summarize it in one sentence.")
msg.tool_calls


[{'name': 'requests_get',
  'args': {'url': 'https://petstore.swagger.io/v2/pet/42'},
  'id': 'call_O2f9YzZVrj4ljoGUDlSUTq7h',
  'type': 'tool_call'}]

In [90]:
# Helper: execute tool calls locally (single-step execution).
from langchain_core.messages import AIMessage, ToolMessage

def execute_tool_calls(ai_message: AIMessage, tools):
    tool_map = {t.name: t for t in tools}
    tool_messages = []
    for call in ai_message.tool_calls or []:
        name = call["name"]
        args = call.get("args") or {}
        result = tool_map[name].invoke(args)
        tool_messages.append(ToolMessage(content=str(result), tool_call_id=call["id"]))
    return tool_messages

tool_messages = execute_tool_calls(msg, openapi_tools)
tool_messages[:1]


[ToolMessage(content='{"code":1,"type":"error","message":"Pet not found"}', tool_call_id='call_O2f9YzZVrj4ljoGUDlSUTq7h')]

### Routing (modern tool-calling)

In lesson 3, we showed an example of **function calling** deciding between two candidate functions.

In LangChain v1 (and OpenAI's current API), the modern pattern is **tool calling**:
- You bind tools with `llm.bind_tools([...])`
- The model returns `AIMessage.tool_calls`
- Your application executes the tool(s) and optionally feeds results back to the model


In [91]:
from langchain_core.utils.function_calling import convert_to_openai_tool

tools = [search_wikipedia, get_current_temperature]
openai_tools_schema = [convert_to_openai_tool(t) for t in tools]
openai_tools_schema


[{'type': 'function',
  'function': {'name': 'search_wikipedia',
   'description': 'Run Wikipedia search and get page summaries.',
   'parameters': {'properties': {'query': {'type': 'string'}},
    'required': ['query'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'get_current_temperature',
   'description': 'Fetch current temperature for given coordinates.',
   'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
      'type': 'number'},
     'longitude': {'description': 'Longitude of the location to fetch weather data for',
      'type': 'number'}},
    'required': ['latitude', 'longitude'],
    'type': 'object'}}}]

In [92]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None}).bind_tools(tools)


In [93]:
model.invoke("What is the weather in San Francisco right now?")


AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 184, 'total_tokens': 283, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ctj6yOXzix1oRI97PBp6XiKqiN0op', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b8119-66b4-7f22-bcff-831e695eadde-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': 'call_0yW1wo0YVWMfEUGqA5eDF2jT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 184, 'output_tokens': 99, 'total_tokens': 283, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [94]:
model.invoke("What is LangChain?")


AIMessage(content='Short answer\n- LangChain is an open-source framework for building applications that use large language models (LLMs). It provides reusable components and patterns so you can combine LLMs with data, tools, and logic to build production-ready apps (chatbots, RAG systems, agents, summarizers, etc.).\n\nCore ideas and components\n- Chains: Composable pipelines that connect prompts, LLM calls, parsing, and other steps into workflows.\n- Prompts & PromptTemplates: Structured prompt management (templates, partials, formatting helpers).\n- Models & Adapters: Unified interface for calling many LLM providers (OpenAI, Anthropic, Hugging Face, local models, etc.).\n- Agents & Tools: Agents plan actions and call external tools or APIs dynamically (search, calculators, file loaders, custom functions).\n- Retrievers & Vector Stores: Retrieval layers for RAG — nearest-neighbor search over embeddings with connectors to stores like FAISS, Pinecone, Milvus, Weaviate, Chroma, etc.\n- M

In [95]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful but sassy assistant."),
    ("user", "{input}"),
])

chain = prompt | model


In [96]:
result = chain.invoke({"input": "What is the weather in SF right now?"})
type(result), result.tool_calls


(langchain_core.messages.ai.AIMessage,
 [{'name': 'get_current_temperature',
   'args': {'latitude': 37.7749, 'longitude': -122.4194},
   'id': 'call_x2N78myfq2eHT9MPCCREvRI7',
   'type': 'tool_call'}])

In [97]:
# Execute the tool call(s) produced by the model.
tool_messages = execute_tool_calls(result, tools)
tool_messages


[ToolMessage(content='The current temperature is 13.2°C', tool_call_id='call_x2N78myfq2eHT9MPCCREvRI7')]

In [98]:
# (Optional) Send tool results back to the model to get a final natural-language answer.
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are a helpful but sassy assistant."),
    HumanMessage(content="What is the weather in SF right now?"),
    result,
    *tool_messages,
]

final_llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None})
final_llm.invoke(messages)


AIMessage(content='Right now in San Francisco it’s 13.2°C (about 55.8°F). Pretty brisk — sweater weather. Want more details (current conditions, humidity, wind, or a short-term forecast)?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 308, 'prompt_tokens': 78, 'total_tokens': 386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ctj7OHU1TJ7Gix9qQKlxrFeDpp7B2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b8119-cbe4-7bc3-92d1-b4bcec95f9a8-0', usage_metadata={'input_tokens': 78, 'output_tokens': 308, 'total_tokens': 386, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

In [99]:
# A simple "router": if there are tool calls, execute them; otherwise return the text.
def route_once(ai_message, tools):
    if getattr(ai_message, "tool_calls", None):
        tool_messages = execute_tool_calls(ai_message, tools)
        return {"ai_message": ai_message, "tool_messages": tool_messages}
    return {"ai_message": ai_message, "tool_messages": []}

routed = route_once(chain.invoke({"input": "What is LangChain?"}), tools)
routed["ai_message"].content


'Short answer: LangChain is an open-source framework for building applications that use large language models — especially apps that need to connect LLMs to data, tools, and multi-step logic (think RAG, chatbots with memory, agentic workflows, etc.).\n\nLonger answer (but still not a novel):\n- Purpose: Make it easy to compose LLM calls, retrieval, tool use, state/memory, and orchestration into reliable applications rather than one-off prompts.\n- Languages: Official SDKs for Python and JavaScript/TypeScript (most tooling and examples are in those).\n\nCore building blocks\n- LLM wrappers: unified interfaces for model providers (OpenAI, Anthropic, Cohere, local models, etc.).\n- Prompts & PromptTemplates: reusable prompt templates and prompt management.\n- Chains: composable pipelines of steps (LLM → logic → retrieval → LLM, etc.).\n- Agents & Tools: agents that plan which tools to call (web search, calculators, DB queries) and execute multi-step tasks.\n- Memory: manage conversational

In [100]:
routed = route_once(chain.invoke({"input": "What is the weather in San Francisco right now?"}), tools)
routed["ai_message"].tool_calls


[{'name': 'get_current_temperature',
  'args': {'latitude': 37.7749, 'longitude': -122.4194},
  'id': 'call_twwbb1ZxEWTDH8GbTVSxXHKZ',
  'type': 'tool_call'}]

In [101]:
# Run the tool(s) and get a final answer in one helper.
def answer_with_tools(user_input: str, tools):
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None}).bind_tools(tools)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful but sassy assistant."),
        ("user", "{input}"),
    ])
    first = (prompt | llm).invoke({"input": user_input})
    tool_messages = execute_tool_calls(first, tools)
    if not tool_messages:
        return first.content

    final_llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None})
    messages = [
        SystemMessage(content="You are a helpful but sassy assistant."),
        HumanMessage(content=user_input),
        first,
        *tool_messages,
    ]
    return final_llm.invoke(messages).content

answer_with_tools("What is the weather in San Francisco right now?", tools)


'It’s 13.2°C (55.8°F) in San Francisco right now. Chilly by Bay standards—you might want a light jacket. Want an hourly forecast, rain chances, or wind info?'

In [102]:
answer_with_tools("What is LangChain?", tools)


'Short answer: LangChain is an open-source framework for building applications that use large language models (LLMs). It doesn’t replace LLMs — it helps you orchestrate them, connect them to data and tools, and build reliable apps like chatbots, retrieval-augmented generation (RAG), agents, and automation workflows.\n\nWhat LangChain gives you\n- Chains: Compose model calls, prompt templates, data transforms, and logic into reusable pipelines.\n- Prompts & PromptTemplates: Manage and templatize prompts safely and consistently.\n- Memory: Keep conversational state across interactions (short-term, long-term, or custom).\n- Agents & Tools: Let models decide what actions to take (call APIs, run code, query DBs).\n- Document loaders & text splitters: Ingest and prepare text from PDFs, web pages, etc.\n- Embeddings & Vectorstores: Create embeddings and store/search them (Chroma, Pinecone, Milvus, Weaviate, Supabase, etc.).\n- Retrievers & RAG: Combine retrieval with generation to ground answ

In [103]:
answer_with_tools("hi!", tools)


'Hey! What’s up? I’m here and ready — what can I do for you today? (Ask me to summarize something, write an email, solve a problem, or just spill the tea.)'

In [ ]:
answer_with_tools("Search Wikipedia for LangChain and give me one sentence.", tools)


In [ ]:
answer_with_tools("What's the temperature in London right now?", tools)


In [ ]:
# You can also inspect the raw tool calls:
llm_tools = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None}).bind_tools(tools)
raw = (prompt | llm_tools).invoke({"input": "What's the weather in SF right now?"})
raw.tool_calls


In [ ]:
execute_tool_calls(raw, tools)


In [ ]:
# Final answer after tool execution:
user_input = "What's the weather in SF right now?"
tool_msgs = execute_tool_calls(raw, tools)
final_llm = ChatOpenAI(model="gpt-5-mini", temperature=0, disabled_params={"stop": None})
final_llm.invoke([
    SystemMessage(content="You are a helpful but sassy assistant."),
    HumanMessage(content=user_input),
    raw,
    *tool_msgs,
]).content
